# Preparação de dados para processamento NLP

In [108]:
import pandas as pd
from bleach.sanitizer import Cleaner
import regex as re

## 1. Importando dados da análise

In [109]:
PARQUET_MPO_SELECIONADOS = '../../Data/Processed/mpo_selecionados.parquet'

df_mpo = pd.read_parquet(PARQUET_MPO_SELECIONADOS)

In [110]:
df_mpo

,ID,Level,MpoIdentificador,MpoAreaEletrica,MpoCentro,MpoLigadoDesligado,Texto,Título
0,161279,1,IO-PM.6MD,6MD,CNOS,Não definido,"<div><p style=""margin-top:0pt; margin-bottom:6...",OBJETIVO
1,161280,2,IO-PM.6MD,6MD,CNOS,Não definido,"<div><p style=""margin-top:6pt; margin-left:34p...",CONSIDERAÇÕES GERAIS
2,161281,3,IO-PM.6MD,6MD,CNOS,Não definido,"<div><p style=""margin-top:0pt; margin-bottom:0...",PREPARAÇÃO PARA MANOBRAS EM LINHAS DE TRANSMISSÃO
3,161282,3.1,IO-PM.6MD,6MD,CNOS,Não definido,"<div><p style=""margin-top:0pt; margin-bottom:0...",LT 230 kV Coletora Porto Velho / Porto Velho C...
4,161283,3.1.1,IO-PM.6MD,6MD,CNOS,Desligar,"<div><h4 style=""margin-top:12pt; margin-left:4...",Desenergização da LT 230 kV Coletora Porto Vel...
...,...,...,...,...,...,...,...,...
5451,166760,5.5.3,IO-PM.SSE,SSE,CNOS,Ligar,"<div><table style=""margin-left:3.5pt; border-c...",Energização do banco de autotransformadores de...
5452,166761,5.6,IO-PM.SSE,SSE,CNOS,Não definido,"<div><p style=""margin-top:0pt; margin-bottom:0...",SE IBIÚNA – BANCO DE AUTOTRANSFORMADORES DE 50...
5453,166762,5.6.1,IO-PM.SSE,SSE,CNOS,Desligar,"<div><table style=""margin-left:3.5pt; border:2...",Desenergização de um Banco de transformador 50...
5454,166763,5.6.2,IO-PM.SSE,SSE,CNOS,Ligar,"<div><table style=""width:465.4pt; margin-left:...",Energização de um Banco de transformador 500/3...


## 2. Limpando elementos HTML da coluna Texto

In [124]:
def limpa_ruido_texto(texto):
    texto = re.sub('(&#xa0;)', ' ', texto)
    texto = re.sub('(\\xa0)', ' ', texto)
    texto = re.sub('color:#ffffff">', ' ', texto)
    texto = re.sub('</span>', ' </span>', texto)
    texto = re.sub('\uf0b7', ' ', texto)
    texto = re.sub('\uf044', ' ', texto)
    return texto

In [125]:
# Retira alguns ruídos preparando texto para a limpeza de tags html
df_mpo['Texto'] = df_mpo['Texto'].apply(lambda x: limpa_ruido_texto(x))

In [113]:
cleaner = Cleaner(tags=[], attributes=[], styles=[], protocols=[], strip=True, strip_comments=True, filters=None)

# Retira todas as tags html do texto
df_mpo['Texto'] = df_mpo['Texto'].apply(lambda x: cleaner.clean(x))

### Checando alguns exemplos

In [126]:
df_mpo['Texto'].iloc[10]

'   1.   OBJETIVO  2. .................................................. CONSIDERAÇÕES GERAIS  3.                PREPARAÇÃO PARA MANOBRAS EM LINHAS DE TRANSMISSÃO  3.1.                LT 230 kV Coletora Porto Velho / Porto Velho C1 e/ou C2  3.1.1.                Desenergização da LT 230 kV Coletora Porto Velho / Porto Velho C1 ou C2  3.1.2.                Energização da LT 230 kV Coletora Porto Velho / Porto Velho C1 ou C2  3.1.3.                Energização dos dois circuitos da LT 230 kV Coletora Porto Velho / Porto Velho (C1 e C2)  3.2.                LT 500 kV Santo Antônio Margem Direita / Coletora Porto Velho C1 ou LT 500 kV Santo Antônio Leito do Rio / Coletora Porto Velho C2 (SISTÊMICO)  3.2.1.                Desenergização da LT 500 kV Santo Antônio Margem Direita / Coletora Porto Velho C1 ou LT 500 kV Santo Antônio Leito do Rio / Coletora Porto Velho C2  3.2.2.                Energização da LT 500 kV Santo Antônio Margem Direita / Coletora Porto Velho C1 ou da LT 500 kV Santo 

In [127]:
df_mpo[df_mpo['MpoLigadoDesligado']=='Ligar']['Texto'].iloc[200]

'3.4.2.1.        Energização da LT 500 kV Ribeirãozinho / Cuiabá C1 (circuito 1) a)      A partir da SE Ribeirãozinho (sentido normal) Estando um dos reatores de linha da LT 500 kV Ribeirãozinho / Cuiabá C1 indisponível, essa LT não poderá ser energizada. 1 COSR-NCO COSR-NCO  ITE (COS-State Grid) Energizar a LT 500 kV Ribeirãozinho / Cuiabá C1,  pelo terminal da   SE Ribeirãozinho. - Reatores de linha da LT conectados. - SE Ribeirãozinho  ≤   537 kV . - SE Cuiabá  ≤   531 kV . - CE da SE Coxipó com folga para absorção de  70 Mvar. Observação: Preferencialmente , manter as tensões nos barramentos de 230 kV das SEs Rio Verde, Barra do Peixe e Ribeirãozinho  em valor inferior a 235 kV . Fechar o terminal da LT 500 kV Ribeirãozinho / Cuiabá C1,  na SE Cuiabá .   V  ≤  100 kV   \uf071   ≤  30 0   b)     A partir da SE Cuiabá (sentido inverso) Estando um dos reatores de linha da LT 500 kV Ribeirãozinho / Cuiabá C1  indisponível , essa LT não poderá ser energizada. 1 COSR-NCO COSR-NCO  ITE (C

In [128]:
df_mpo[df_mpo['MpoLigadoDesligado']=='Desligar']['Texto'].iloc[3]

'1 COSR-NCO COSR-NCO --- Atender previamente as restrições de operação impostas por esta indisponibilidade. Conforme  IO-OC.6MD . 2 Sendo a desenergização do  C4 ,  primeiramente  adotar o procedimento a seguir: 2.1 COSR-NCO COSR-NCO Santo Antônio Abrir ou manter aberto o terminal de 500 kV do Transformador TF13  525 /23 0 kV da SE Coletora Porto Velho . Conforme   item específico desta  IO-PM.6MD . 3 COSR-NCO COSR-NCO Santo Antônio Abri r o terminal da LT 500 kV Santo Antônio Margem Esquerda / Coletora Porto Velho C3 ou C4,  na UHE Santo Antônio . SE Coletora Porto Velho ≤  550 kV . Desenergizar a LT 500 kV Santo Antônio Margem Esquerda / Coletora Porto Velho C3 ou C4,  pelo terminal da SE Coletora Porto Velho .     '

## 3. Gerando parquet para uso na operação dos dados

In [129]:
PARQUET_MPO_PREPARADOS = '../../Data/Processed/mpo_preparados.parquet'

df_mpo.to_parquet(PARQUET_MPO_PREPARADOS)